In [4]:
!pip install matplotlib

In [6]:
import qiskit
from qiskit import QuantumCircuit
import numpy as np

# Function to calculate rotation angle
def calc_angle(prob):
    #Calculate the angle using the formula
    return 2 * np.arccos(np.sqrt(prob))

def create_quantum_circuit(null_prior, null_probs, positive_probs):
    # Calculate angles based on the provided probabilities
    theta_y = calc_angle(null_prior)
    null_rotations = [calc_angle(conditional) for conditional in null_probs]
    pos_rotations = [calc_angle(conditional) for conditional in positive_probs]
    
    # Initialize the quantum circuit with n qubits and n classical bits
    assert(len(null_probs) == len(positive_probs))
    n = len(null_probs) #Number of features
    circ = QuantumCircuit(n+1)
    
    # Implementing the gates based on the calculated thetas
    circ.ry(theta_y, 0)  # Encode P(y=0) into qubit 0
    for i in range(n):
        circ.cry(null_rotations[i], 0, i+1)
    circ.x(0)  # Flip y to represent y=1
    for i in range(n):
        circ.cry(pos_rotations[i], 0, i+1) # Now for y=1, encode P(x1|y=1)
    circ.x(0)  # Reset y back
    
    # Measure 
    circ.measure_all()  # Measure both the label and feature qubits
    
    return circ

# Example usage
P_y0 = 0.6
P_xn_given_y0 = [0.7, 0.3, 0.1, 0.9]
P_xn_given_y1 = [0.2, 0.6, 0.8, 0.15]
circ = create_quantum_circuit(P_y0, P_xn_given_y0, P_xn_given_y1)

# To draw the circuit using Matplotlib, ensure you have matplotlib installed and your environment supports plotting
circ.draw()


┌────────────┐                                                        »
   q_0: ┤ Ry(1.3694) ├──────■─────────────■─────────────■─────────────■───────»
        └────────────┘┌─────┴──────┐      │             │             │       »
   q_1: ──────────────┤ Ry(1.1593) ├──────┼─────────────┼─────────────┼───────»
                      └────────────┘┌─────┴──────┐      │             │       »
   q_2: ────────────────────────────┤ Ry(1.9823) ├──────┼─────────────┼───────»
                                    └────────────┘┌─────┴──────┐      │       »
   q_3: ──────────────────────────────────────────┤ Ry(2.4981) ├──────┼───────»
                                                  └────────────┘┌─────┴──────┐»
   q_4: ────────────────────────────────────────────────────────┤ Ry(0.6435) ├»
                                                                └────────────┘»
meas: 5/══════════════════════════════════════════════════════════════════════»
                                                                              »
«        ┌───┐                                                        ┌───┐ ░ »
«   q_0: ┤ X ├──────■─────────────■─────────────■─────────────■───────┤ X ├─░─»
«        └───┘┌─────┴──────┐      │             │             │       └───┘ ░ »
«   q_1: ─────┤ Ry(2.2143) ├──────┼─────────────┼─────────────┼─────────────░─»
«             └────────────┘┌─────┴──────┐      │             │             ░ »
«   q_2: ───────────────────┤ Ry(1.3694) ├──────┼─────────────┼─────────────░─»
«                           └────────────┘┌─────┴──────┐      │             ░ »
«   q_3: ─────────────────────────────────┤ Ry(0.9273) ├──────┼─────────────░─»
«                                         └────────────┘┌─────┴──────┐      ░ »
«   q_4: ───────────────────────────────────────────────┤ Ry(2.3462) ├──────░─»
«                                                       └────────────┘      ░ »
«meas: 5/═════════════════════════════════════════════════════════════════════»
«                                                                             »
«        ┌─┐            
«   q_0: ┤M├────────────
«        └╥┘┌─┐         
«   q_1: ─╫─┤M├─────────
«         ║ └╥┘┌─┐      
«   q_2: ─╫──╫─┤M├──────
«         ║  ║ └╥┘┌─┐   
«   q_3: ─╫──╫──╫─┤M├───
«         ║  ║  ║ └╥┘┌─┐
«   q_4: ─╫──╫──╫──╫─┤M├
«         ║  ║  ║  ║ └╥┘
«meas: 5/═╩══╩══╩══╩══╩═
«         0  1  2  3  4

In [18]:
import numpy as np
import os
import gzip
import requests
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import train_test_split
from skimage.measure import block_reduce

# Function to download MNIST dataset
def download_mnist():
    base_url = 'http://yann.lecun.com/exdb/mnist/'
    files = ['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz',
             't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']

    for file in files:
        url = base_url + file
        r = requests.get(url, stream=True)
        with open(file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=128):
                f.write(chunk)

# Function to load and pool MNIST dataset
def load_mnist_with_pooling():
    def load_mnist_images(filename):
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        images = data.reshape(-1, 28, 28)

        # Apply pooling
        # Using block_reduce to downsample the image
        pooled_images = np.array([block_reduce(image, block_size=(9, 9), func=np.mean) for image in images])

        # Reshape back to 2D if necessary
        return pooled_images.reshape(images.shape[0], -1)

    def load_mnist_labels(filename):
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        return data

    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

    return np.concatenate([X_train, X_test]), np.concatenate([y_train, y_test])

# Download and load the dataset
download_mnist()
X, y = load_mnist_with_pooling()

# Convert labels to integers
y = y.astype(np.int8)

# Filter out only 0s and 1s
X_filtered = X[np.logical_or(y == 0, y == 1)]
y_filtered = y[np.logical_or(y == 0, y == 1)]

# Binarize the data
binarizer = Binarizer(threshold=128)
X_binarized = binarizer.transform(X_filtered)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_binarized, y_filtered, test_size=0.2, random_state=42)

# Calculate class priors (P(y))
class_priors = [np.mean(y_train == c) for c in [0, 1]]

# Calculate conditional probabilities P(xi|y)
conditional_probs = []
for c in [0, 1]:
    class_features = X_train[y_train == c]
    class_conditional_probs = np.mean(class_features, axis=0)
    conditional_probs.append(class_conditional_probs)

print(class_priors)
print(conditional_probs)


[0.4652401894451962, 0.5347598105548038]
[array([0.        , 0.00036357, 0.        , 0.        , 0.        ,
       0.06380658, 0.03562989, 0.        , 0.        , 0.05108162,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ]), array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.05677685, 0.        , 0.        , 0.        , 0.00047446,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])]


In [37]:
import numpy as np

# Import Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import Aer


def calc_angle(prob):
    return 2 * np.arccos(np.sqrt(prob))

def create_quantum_circuit(class_priors, conditional_probs):
    assert(len(class_priors) == 2 and len(conditional_probs) == 2)
    assert(len(conditional_probs[0]) == len(conditional_probs[1]))

    n = len(conditional_probs[0])
    circ = QuantumCircuit(n + 1, n + 1)

    theta_y0 = calc_angle(class_priors[0])
    circ.ry(theta_y0, 0)

    for i in range(n):
        theta = calc_angle(conditional_probs[0][i])
        circ.cry(theta, 0, i + 1)

    circ.x(0)

    for i in range(n):
        theta = calc_angle(conditional_probs[1][i])
        circ.cry(theta, 0, i + 1)

    circ.x(0)

    circ.measure(range(n + 1), range(n + 1))

    return circ

# Example probabilities
# class_priors = [0.5, 0.5]
# conditional_probs = [
#     [0.7, 0.3, 0.1, 0.9],
#     [0.2, 0.6, 0.8, 0.15]
# ]

circ = create_quantum_circuit(class_priors, conditional_probs)

# Select the Aer simulator
simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit on the simulator
job = execute(circ, simulator, shots=1024)

# Grab the results from the job
result = job.result()

# Get the counts
counts = result.get_counts(circ)

print(counts)


ModuleNotFoundError: No module named 'qiskit.providers.aer'